<a href="https://colab.research.google.com/github/e3la/i2dc/blob/main/i2dc_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to i2dc an instagram to digitalcommons tool!
## This code is the result of vibe coding and was built mostly using gemini.

This Colab notebook transforms an Instagram zip archive into structured zip packages for upload to institutional repositories using Digital Commons. You will be guided guided through the process, with a few questions to configure your export before processing your data.

A separate ZIP packages for your Posts, Reels, and Stories, will be generated at your request each containing the media files and a corresponding Excel metadata file ready for review and batch upload.

First you provide the archive zip you got from instagram, by pressing the play button the left of the box below and follow the steps. When done you can close this cell with the carrot.

> Add blockquote





In [ ]:
#@title Upload your zip from instagram
#@markdown You can upload directly or connect to google drive and put your zips in MyDrive/i2dc/.

from google.colab import files, drive
import os
import shutil
import zipfile
import json
import re

# This variable will be used by the next cells
zip_filepath = None

def setup_zip_file_interactive():
    """
    Interactively asks the user how they want to provide their Instagram archive ZIP file.
    """
    while True:
        print("\n--- 📂 How would you like to provide the Instagram ZIP file? ---")
        method = input(
            "1. Upload from my computer\n"
            "2. Use Google Drive\n"
            "Enter choice (1 or 2): "
        ).strip()

        if method == '1':
            print("\nPlease click 'Choose Files' and select your Instagram ZIP archive.")
            uploaded = files.upload()
            if uploaded:
                filename = list(uploaded.keys())[0]
                if filename.lower().endswith('.zip'):
                    print(f"✔️ Successfully uploaded: {filename}")
                    return os.path.join('/content', filename)
                else:
                    print(f"❌ ERROR: The uploaded file '{filename}' is not a ZIP file. Please try again.")
            else:
                print("❌ No file was uploaded. Please try again.")

        elif method == '2':
            print("\nSelected: Use Google Drive.")
            print("Connecting to your Google Drive...")
            # 'force_remount=True' ensures a fresh connection every time.
            drive.mount('/content/drive', force_remount=True)

            # This is the standard path where the script will look for your files.
            gdrive_path = "/content/drive/MyDrive/i2dc/"
            print(f"Searching for .zip files in your Google Drive at: '{gdrive_path}'")

            if not os.path.isdir(gdrive_path):
                print(f"❌ ERROR: The folder '{gdrive_path}' was not found.")
                print("Please create a folder named 'i2dc' in your 'My Drive' and place your ZIP file inside it, then run this cell again.")
                return None

            zip_files_found = [os.path.join(gdrive_path, f) for f in os.listdir(gdrive_path) if f.lower().endswith('.zip')]

            if not zip_files_found:
                print(f"❌ No .zip files found in '{gdrive_path}'. Please add your file and try again.")
                return None
            elif len(zip_files_found) == 1:
                chosen_filepath = zip_files_found[0]
                print(f"✔️ Found one ZIP file: '{os.path.basename(chosen_filepath)}'")
                return chosen_filepath
            else:
                print(f"\nMultiple .zip files found. Please choose one:")
                while True:
                    for i, filepath in enumerate(zip_files_found):
                        print(f"  {i+1}. {os.path.basename(filepath)}")
                    try:
                        choice_str = input(f"Enter the number of the file you want to use (1-{len(zip_files_found)}): ")
                        choice_int = int(choice_str)
                        if 1 <= choice_int <= len(zip_files_found):
                            chosen_filepath = zip_files_found[choice_int - 1]
                            print(f"✔️ You selected: '{os.path.basename(chosen_filepath)}'")
                            return chosen_filepath
                        else:
                            print(f"❌ Invalid number. Please enter a number between 1 and {len(zip_files_found)}.")
                    except ValueError:
                        print("❌ Invalid input. Please enter a number.")
        else:
            print("❌ Invalid choice. Please enter 1 or 2.")

# Run the interactive function to get the file path
zip_filepath = setup_zip_file_interactive()

if zip_filepath:
    print(f"\n✅ File ready at: {zip_filepath}")

# --- Global Constants needed for this step ---
BASE_DIR = "/content"
EXTRACTED_DATA_DIR = os.path.join(BASE_DIR, "extracted_data")
instagram_handle = "unknown_user" # Default value

# --- Helper function updated with your robust JSON parsing logic ---
def extract_instagram_handle(personal_info_path):
    """
    Reads the 'personal_information.json' file to find the user's
    Instagram username using a safe, step-by-step parsing method.
    """
    try:
        with open(personal_info_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # --- This is your improved, safer navigation logic ---
        username_value = None
        if isinstance(data, dict) and "profile_user" in data and isinstance(data["profile_user"], list) and len(data["profile_user"]) > 0:
            # Assuming the relevant data is in the first element of the profile_user list
            profile_info = data["profile_user"][0]
            if isinstance(profile_info, dict) and "string_map_data" in profile_info and isinstance(profile_info["string_map_data"], dict):
                string_data = profile_info["string_map_data"]
                if "Username" in string_data and isinstance(string_data["Username"], dict) and "value" in string_data["Username"]:
                    username_value = string_data["Username"]["value"]
        # --- End of your logic ---

        if username_value:
            return username_value
        else:
            print("  -> Username key not found in the expected structure within the file.")
            return "unknown_user"

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"  -> Could not read or parse the JSON file. Reason: {e}")
        return "unknown_user"

# --- Main logic for this cell ---
print("⚙️ Processing your file to find your username...")

if 'zip_filepath' not in globals() or not zip_filepath or not os.path.exists(zip_filepath):
    print("\n❌ Process stopped. A valid ZIP file was not provided in the previous step.")
    print("Please go back to Step 1 and provide a file.")
else:
    # Extract the archive
    print(f"--- 📂 Extracting {os.path.basename(zip_filepath)} ---")
    if os.path.exists(EXTRACTED_DATA_DIR):
        shutil.rmtree(EXTRACTED_DATA_DIR)
    os.makedirs(EXTRACTED_DATA_DIR, exist_ok=True)
    try:
        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            zip_ref.extractall(EXTRACTED_DATA_DIR)
        print("✔️ Successfully extracted archive.")
    except Exception as e:
        print(f"❌ ERROR during extraction: {e}")

    # Find the correct path to the JSON file by checking common locations
    path_to_check = None
    path1 = os.path.join(EXTRACTED_DATA_DIR, 'your_instagram_activity', 'account_information', 'personal_information.json')
    path2 = os.path.join(EXTRACTED_DATA_DIR, 'personal_information', 'personal_information.json')
    # Add your specific path as a fallback, just in case
    path3 = "/content/extracted_data/personal_information/personal_information/personal_information.json"


    if os.path.exists(path1):
        path_to_check = path1
    elif os.path.exists(path2):
        path_to_check = path2
    elif os.path.exists(path3):
        path_to_check = path3

    if path_to_check:
        print(f"✔️ Found metadata file at: {path_to_check}")
        instagram_handle = extract_instagram_handle(path_to_check)
    else:
        print("❌ Could not find 'personal_information.json' in any standard location within the ZIP.")


    if instagram_handle != "unknown_user":
        print(f"\n✅ Username Found: @{instagram_handle}")
        print("\n➡️ You can now run the configuration cell below.")
    else:
        print("\n⚠️ Could not automatically find your username. Will use a placeholder.")
        print("\n➡️ You can now run the configuration cell below.")


--- 📂 How would you like to provide the Instagram ZIP file? ---


In [ ]:
#@title Configure Your Instagram to Digital Commons Export
#@markdown This form lets you customize the output. Your actual username has been found and is used in the examples below.

# --- Display dynamic examples first ---
if 'instagram_handle' not in globals() or instagram_handle == 'unknown_user':
    ig_handle_for_examples = "your_username"
    print("⚠️ Username not found in the previous step. Using generic examples.")
else:
    ig_handle_for_examples = instagram_handle

print("--- Title Format Examples (using your username) ---")
print(f"1. Default: Instagram Post by {ig_handle_for_examples} on 2024-08-26")
print(f"2. Simple:  {ig_handle_for_examples} | Post | 2024-08-26")
print(f"3. Alt:     Post by {ig_handle_for_examples} (2024-08-26)")
print("-" * 50)


#@markdown ### **1. Content to Process**
#@markdown Select which types of Instagram content you want to include in the export.
process_posts = True #@param {type:"boolean"}
process_reels = True #@param {type:"boolean"}
process_stories = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### **2. Title Formatting**
#@markdown Choose a title format from the examples printed above.
title_format_choice = "User | Type | Date" #@param ["Default (Type by User on Date)", "User | Type | Date", "Type by User (Date)", "Date - User - Type", "Custom Format..."]
#@markdown If you chose "Custom Format...", define it below using `{username}`, `{doc_type}`, `{doc_type_short}`, and `{date}`.
custom_title_template = "" #@param {type:"string"}

#@markdown ---
#@markdown ### **3. Metadata Columns**
#@markdown Enter the metadata columns for the final Excel file, separated by commas.
#@markdown If you want them, add a comma and any of these: like_count, comments_disabled, latitude, longitude, original_filename, source_file_path
metadata_columns_str = "title, publication_date, abstract, keywords, document_type, fulltext_url, additional_files, instagram_username" #@param {type:"string"}

print("\n✅ Configuration loaded.")
print("\n➡️ All settings are ready. Run the final cell below to process your data!")

In [ ]:
#@title Get your zips
#@markdown Run this and get your downloads.

# --- Initial Setup ---
!pip install ftfy -q
print("✔️ 'ftfy' library is installed and ready to help clean up text.")

# --- Core Library Imports ---
import os, shutil, zipfile, json, re
from datetime import datetime
import pandas as pd
from google.colab import files

try:
    import ftfy
    FTFY_AVAILABLE = True
except ImportError:
    FTFY_AVAILABLE = False

# --- Global Constants ---
BASE_DIR = "/content"
EXTRACTED_DATA_DIR = os.path.join(BASE_DIR, "extracted_data")
BATCHUP_DIR = os.path.join(BASE_DIR, "batchup")
INSTAGRAM_ACTIVITY_FOLDER_NAME = "your_instagram_activity"

# ==============================================================================
# HELPER FUNCTIONS (Most are already defined, but included for completeness)
# ==============================================================================
def cleanup_content_directory(filename_to_keep=None):
    print("🧹 Cleaning up the workspace for a fresh start...")
    items_to_preserve = ["drive", "sample_data"]
    if filename_to_keep:
        items_to_preserve.append(os.path.basename(filename_to_keep))
    for item in os.listdir(BASE_DIR):
        if item in items_to_preserve: continue
        item_path = os.path.join(BASE_DIR, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path): os.unlink(item_path)
            elif os.path.isdir(item_path): shutil.rmtree(item_path)
        except Exception as e:
            print(f"  ⚠️ Could not delete {item_path}. Reason: {e}")
    # Don't print "Workspace is clean." here as it's not the goal of the main script

def extract_hashtags(text):
    if not isinstance(text, str): return ""
    hashtags = re.findall(r"#(\w+)", text)
    return ", ".join(hashtags)

def fix_json_encoding(media_json_dir):
    if not FTFY_AVAILABLE:
        print("\nℹ️ `ftfy` library not available. Skipping automatic text & emoji fixing.")
        return
    print("\n--- 🔎 Scanning and fixing text encoding in JSON files ---")
    if not os.path.isdir(media_json_dir):
        print(f"❌ ERROR: Media JSON directory not found at: {media_json_dir}")
        return
    total_files_changed = 0
    total_fields_fixed = 0
    def _fix_text_in_obj(obj, key):
        nonlocal total_fields_fixed
        original_text = obj.get(key)
        if isinstance(original_text, str) and original_text:
            fixed_text = ftfy.fix_text(original_text)
            if fixed_text != original_text:
                obj[key] = fixed_text; total_fields_fixed += 1; return True
        return False
    for filename in os.listdir(media_json_dir):
        if not filename.lower().endswith('.json'): continue
        json_path = os.path.join(media_json_dir, filename)
        try:
            with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
            file_was_changed = False
            if 'reels.json' in filename or 'stories.json' in filename:
                key = 'ig_reels_media' if 'reels' in filename else 'ig_stories'
                if key in data and isinstance(data.get(key), list):
                    for item in data[key]:
                        for media_item in item.get('media', [item]):
                            if isinstance(media_item, dict) and _fix_text_in_obj(media_item, 'title'): file_was_changed = True
            elif 'posts_1.json' in filename:
                if isinstance(data, list):
                    for post in data:
                        if isinstance(post, dict):
                            if _fix_text_in_obj(post, 'title'): file_was_changed = True
                            for media_item in post.get('media', []):
                                 if isinstance(media_item, dict) and _fix_text_in_obj(media_item, 'title'): file_was_changed = True
            if file_was_changed:
                total_files_changed += 1
                with open(json_path, 'w', encoding='utf-8') as f: json.dump(data, f, ensure_ascii=False, indent=2)
                print(f"  🔧 Fixed text/emojis in: {filename}")
        except Exception as e:
            print(f"  ⚠️ Could not process {filename}. Reason: {e}")
    print(f"✔️ Text fixing complete. Total fields fixed: {total_fields_fixed} in {total_files_changed} files.")
    print("-" * 50)

# ==============================================================================
# CORE PROCESSING FUNCTION
# ==============================================================================
def process_media_type(media_type, json_filename, username, selected_columns, title_format_choice_tuple):
    print(f"\n{'='*20} PROCESSING: {media_type.upper()} {'='*20}")
    output_dir = os.path.join(EXTRACTED_DATA_DIR, f'{media_type}_export_dc_format')
    if os.path.exists(output_dir): shutil.rmtree(output_dir) # Clean previous runs for this type
    os.makedirs(output_dir, exist_ok=True)
    print(f"✔️ Created output directory: {output_dir}")
    json_path = os.path.join(EXTRACTED_DATA_DIR, INSTAGRAM_ACTIVITY_FOLDER_NAME, 'media', json_filename)
    excel_path = os.path.join(output_dir, f'{media_type}_metadata.xlsx')
    readme_path = os.path.join(output_dir, 'README.txt')
    final_zip_path_in_batchup = os.path.join(BATCHUP_DIR, f"{media_type}.zip")
    package_zip_path_in_output = os.path.join(output_dir, f'{media_type}_package.zip')
    try:
        with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        print(f"ℹ️ Could not find or read {json_filename}. Skipping this media type.")
        return
    items_to_process = []
    if media_type == 'posts': items_to_process = data
    elif media_type == 'reels': items_to_process = data.get('ig_reels_media', [])
    elif media_type == 'stories': items_to_process = data.get('ig_stories', [])
    if not items_to_process:
        print(f"ℹ️ No items found for '{media_type}'. Nothing to process.")
        return
    items_to_process.sort(key=lambda x: x.get('creation_timestamp', float('inf')))
    print(f"✔️ Found {len(items_to_process)} total {media_type} items to process.")
    excel_data_rows = []
    copied_files_for_zip = []
    skipped_counts = {'no_uri': 0, 'web_link': 0, 'file_missing': 0, 'copy_error': 0}
    for item_index, item in enumerate(items_to_process):
        media_list = item.get('media', [item])
        post_level_timestamp = item.get('creation_timestamp', media_list[0].get('creation_timestamp'))
        if not post_level_timestamp: continue
        post_level_caption = item.get('title', media_list[0].get('title', ''))
        for media_index, media_item in enumerate(media_list):
            if not isinstance(media_item, dict): continue
            original_uri = media_item.get('uri')
            if not original_uri or original_uri.startswith(('http://', 'https://')):
                skipped_counts['no_uri'] += 1; continue
            source_media_path = os.path.join(EXTRACTED_DATA_DIR, original_uri)
            if not os.path.exists(source_media_path):
                skipped_counts['file_missing'] += 1; continue
            date_obj = datetime.fromtimestamp(post_level_timestamp)
            ext = os.path.splitext(original_uri)[-1]
            original_filename_base = os.path.basename(original_uri)
            sanitized_basename = re.sub(r'[^a-zA-Z0-9_-]', '_', os.path.splitext(original_filename_base)[0])
            new_media_filename = f"instagram_{username}_{media_type}_{date_obj.strftime('%Y-%m-%d')}_post{item_index+1}_media{media_index+1}_{sanitized_basename}{ext}"
            dest_media_path = os.path.join(output_dir, new_media_filename)
            try:
                shutil.copy2(source_media_path, dest_media_path)
                copied_files_for_zip.append(dest_media_path)
            except Exception as e:
                skipped_counts['copy_error'] += 1; continue
            additional_files_str = ''
            if media_type == 'reels':
                subtitles_uri = media_item.get('media_metadata', {}).get('video_metadata', {}).get('subtitles', {}).get('uri')
                if subtitles_uri and os.path.exists(os.path.join(EXTRACTED_DATA_DIR, subtitles_uri)):
                    source_srt_path = os.path.join(EXTRACTED_DATA_DIR, subtitles_uri)
                    new_srt_filename = os.path.splitext(new_media_filename)[0] + '.srt'
                    dest_srt_path = os.path.join(output_dir, new_srt_filename)
                    shutil.copy2(source_srt_path, dest_srt_path)
                    copied_files_for_zip.append(dest_srt_path)
                    additional_files_str = new_srt_filename
            location_info = media_item.get('location', {})
            row_data = {
                'title': "Placeholder Title", 'publication_date': date_obj.strftime('%Y-%m-%d'),
                'abstract': post_level_caption, 'keywords': extract_hashtags(post_level_caption),
                'document_type': f"Instagram {media_type.capitalize()}", 'fulltext_url': new_media_filename,
                'additional_files': additional_files_str, 'instagram_username': username,
                'like_count': media_item.get('like_count', ''), 'comments_disabled': item.get('comments_disabled', False),
                'latitude': location_info.get('lat', ''), 'longitude': location_info.get('lng', ''),
                'original_filename': original_filename_base, 'source_file_path': original_uri,
                'creation_timestamp': post_level_timestamp,
            }
            excel_data_rows.append(row_data)
    if not excel_data_rows: print(f"ℹ️ No processable media items found for {media_type}."); return
    df = pd.DataFrame(excel_data_rows)
    choice, custom_template = title_format_choice_tuple
    new_titles = []
    for _, row in df.iterrows():
        template_map = {'Default (Type by User on Date)': "{doc_type} by {username} on {date}", 'User | Type | Date': "{username} | {doc_type_short} | {date}", 'Type by User (Date)': "{doc_type_short} by {username} ({date})", 'Date - User - Type': "{date} - {username} - {doc_type_short}", 'Custom Format...': custom_template or "{doc_type} by {username} on {date}"}
        template_str = template_map.get(choice)
        date_str = datetime.fromtimestamp(row['creation_timestamp']).strftime('%Y-%m-%d')
        doc_type_short = row['document_type'].replace("Instagram ", "")
        new_titles.append(template_str.format(username=row['instagram_username'], doc_type=row['document_type'], doc_type_short=doc_type_short, date=date_str))
    df['title'] = new_titles
    df['_date_str'] = df['creation_timestamp'].apply(lambda ts: datetime.fromtimestamp(ts).strftime('%Y-%m-%d'))
    df['_date_rank'] = df.groupby('_date_str').cumcount() + 1
    date_counts = df['_date_str'].value_counts()
    multi_post_dates = date_counts[date_counts > 1].index
    if not multi_post_dates.empty: df['title'] = df.apply(lambda row: f"{row['title']} - {row['_date_rank']} of {date_counts[row['_date_str']]}" if row['_date_str'] in multi_post_dates else row['title'], axis=1)
    df.drop(columns=['_date_str', '_date_rank', 'creation_timestamp'], inplace=True, errors='ignore')
    final_df = pd.DataFrame(df, columns=[col for col in selected_columns if col in df.columns])
    final_df.to_excel(excel_path, index=False, engine='openpyxl')
    print(f"✔️ Metadata for {len(final_df)} items written to {os.path.basename(excel_path)}")
    with open(readme_path, 'w', encoding='utf-8') as f: f.write(f"Instagram {media_type.capitalize()} Export Package\n{'='*40}\n\nHandle: @{username}\nExported: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\nThis package contains {len(final_df)} exported media items and their metadata.\n\n--- SKIPPED ITEMS SUMMARY ---\n  - No media file path in JSON: {skipped_counts['no_uri']}\n  - Item was a web link (not local): {skipped_counts['web_link']}\n  - Local media file was missing: {skipped_counts['file_missing']}\n  - Error copying file: {skipped_counts['copy_error']}\n\nGenerated by instagram2digitalcommons script.\n")
    with zipfile.ZipFile(package_zip_path_in_output, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(excel_path, arcname=os.path.basename(excel_path))
        zipf.write(readme_path, arcname='README.txt')
        for file_path in copied_files_for_zip: zipf.write(file_path, arcname=os.path.basename(file_path))
    if not os.path.exists(BATCHUP_DIR): os.makedirs(BATCHUP_DIR)
    shutil.copy2(package_zip_path_in_output, final_zip_path_in_batchup)
    print(f"✔️ Final package '{os.path.basename(final_zip_path_in_batchup)}' is ready.")
    files.download(final_zip_path_in_batchup)
    print("-" * 50)

# ==============================================================================
# MAIN EXECUTION SCRIPT
# ==============================================================================
def main():
    print("=" * 60)
    print("🚀 Starting the Instagram to Digital Commons Exporter! 🚀")
    print("=" * 60)
    if 'zip_filepath' not in globals() or not zip_filepath or not os.path.exists(zip_filepath):
        print("\n❌ Process stopped. A valid ZIP file was not provided in Step 1.")
        return
    if 'instagram_handle' not in globals():
        print("\n❌ Process stopped. Username was not identified in Step 2.")
        return
    selected_cols = [col.strip() for col in metadata_columns_str.split(',') if col.strip()]
    title_choice_tuple = (title_format_choice, custom_title_template)
    print("--- Settings Confirmed ---")
    print(f"Username: @{instagram_handle}")
    print(f"Title Format: {title_format_choice}")
    print(f"Processing Posts: {process_posts}, Reels: {process_reels}, Stories: {process_stories}")
    print("-" * 26)
    media_json_dir = os.path.join(EXTRACTED_DATA_DIR, INSTAGRAM_ACTIVITY_FOLDER_NAME, 'media')
    fix_json_encoding(media_json_dir)
    if process_posts: process_media_type('posts', 'posts_1.json', instagram_handle, selected_cols, title_choice_tuple)
    if process_reels: process_media_type('reels', 'reels.json', instagram_handle, selected_cols, title_choice_tuple)
    if process_stories: process_media_type('stories', 'stories.json', instagram_handle, selected_cols, title_choice_tuple)
    print("\n🎉 All processing complete! 🎉")
    print(f"Your final ZIP packages can be found in the '{os.path.basename(BATCHUP_DIR)}' folder in the Files panel on the left.")

# --- Run the main function ---
if __name__ == "__main__":
    main()

✔️ 'ftfy' library is installed and ready to help clean up text.
🚀 Starting the Instagram to Digital Commons Exporter! 🚀
--- Settings Confirmed ---
Username: @umsllibraries
Title Format: User | Type | Date
Processing Posts: True, Reels: True, Stories: True
--------------------------

--- 🔎 Scanning and fixing text encoding in JSON files ---
  🔧 Fixed text/emojis in: stories.json
  🔧 Fixed text/emojis in: reels.json
  🔧 Fixed text/emojis in: posts_1.json
✔️ Text fixing complete. Total fields fixed: 262 in 3 files.
--------------------------------------------------

==================== PROCESSING: POSTS ====================
✔️ Created output directory: /content/extracted_data/posts_export_dc_format
✔️ Found 646 total posts items to process.
✔️ Metadata for 860 items written to posts_metadata.xlsx
✔️ Final package 'posts.zip' is ready.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------

==================== PROCESSING: REELS ====================
✔️ Created output directory: /content/extracted_data/reels_export_dc_format
✔️ Found 25 total reels items to process.
✔️ Metadata for 25 items written to reels_metadata.xlsx
✔️ Final package 'reels.zip' is ready.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------

==================== PROCESSING: STORIES ====================
✔️ Created output directory: /content/extracted_data/stories_export_dc_format
✔️ Found 485 total stories items to process.
✔️ Metadata for 450 items written to stories_metadata.xlsx
✔️ Final package 'stories.zip' is ready.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------

🎉 All processing complete! 🎉
Your final ZIP packages can be found in the 'batchup' folder in the Files panel on the left.
